### Facial Emotion Detection
This exmaple demonstrates how to use python libraries to detect facial emotion through a camera feed.
Libraries used:
1. opencv-python: Opencv is an open source library which is useful for computer vision applications such as video analysis, CCTV footage analysis and image analysis.
2. DeepFace : DeepFace is a deep learning facial recognition system created by a research group at Facebook.

#### Flow of script:

Importing libraries cv2 & DeepFace

In [1]:
import cv2 #pip install opencv-python
from deepface import DeepFace #pip install deepface

import csv
import datetime

Loading our classifier which takes file path of the xml file as a parameter
>"haarcascade" is used for detecting faces in a image.
<br>In our case it will be used to detect face in each frame of the video captured.

In [2]:
cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

Open up our webcam using cv2
>We try opening both cameras one by one if we are not able to open any one of the camera, we raise an error.

In [3]:
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError('Cannot Open Camera')
    
# cap=cv2.VideoCapture('path to the video file')

Creating a test.csv file to store data in.
We open a csv file and we create a csv writer.

In [4]:
#working with csv data
file = open('test.csv', 'w', encoding='UTF8', newline='')
writer = csv.writer(file)

writer.writerow(['Time', 'Emotion']) # writing a header to the file

data = [] #initializing an empty list to store data in

Working on each frame of the captured video
>a. We enter a loop which will read the camera feed and flip each frame.
<br><br>b. Frame is analyzed using Deepface and saves values associated with emotion in result.
<br><br>c. Each fram is then converted to gray for better face detection
<br><br>d. The all the faces' details are stored in faces variable.
<br><br>e. For each face in the frame a rectangle is drawn around the face.
<br><br>f. We then put a text above the rectangle indicating the dominant emotion.
<br><br>g. The video is displyed with all the modification applied to each frame like the rectangle and the text.
<br><br>h. There is last piece of code that is there to stop the video capture when 'q' key is pressed.

In [5]:
#to prevent the crashing of application when there is no face in the feed change the enforce_detection to False on 169th line of D:\Python39\Lib\site-packages\deepface\commons\functions.py file. (D:\Python39\ is where my python is installed) 

while True:
    ret, frame = cap.read() #read the camera feed
    if ret:
        frame = cv2.flip(frame,1)#Fliping each frame to mirror the feed for better output video

    
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) # convert the fram to Gray
    
        faces = cascade.detectMultiScale(gray,1.2, 4) # detects faces in the image/frame
        font = cv2.FONT_HERSHEY_SIMPLEX # font of the text to show the emotion

    
        for (x,y,w,h) in faces: # for loop for putting a rectangle around the face
            result = DeepFace.analyze(frame, actions=['emotion']) #analyze the frame and look for emotion attribute and save it in a result
            
            data.append([datetime.datetime.now(), result['dominant_emotion']]) #save dominant_emotion and time of emotion to the list.
            
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 1)
            # parameters (image,top left coords of detection, bottomright coords of detection, color of rectangle, thickness of border)
            
            cv2.putText(
                frame,
                result['dominant_emotion'].capitalize(),
                (x,y-10),
                font, 1, 
                (0,0,255),
                2
            )  #patameters(image, dominant_emotion, position of text, font,size of text, color of text, boldness of text)

    
     
    
    
        cv2.imshow('video', frame) #replace the input frame with the updated one and show it as output.
        if cv2.waitKey(2) & 0xff == ord('q'):
            break
    else:
        break
        
cap.release() #release the camera or stop reading it
cv2.destroyAllWindows() #destroy the window which was opened when camera was started.

writer.writerows(data) #write the data availabe on emotions to the csv file.
file.close()